# 実験を終了する

実験記録をデータガバナンス機能に保存して、実験を終了します。

## 1.実行環境構成を記録する

In [ ]:
%%sh
conda env export -n base > ~/environment.yml
pip freeze > ~/requirements.txt

## 2. 高性能実験環境から実験記録を取得する

高性能実験環境を利用している場合には、以下のセルを実行してください。

In [ ]:
# !chmod -R 755 /home/jovyan/input_data/*/.git/annex/objects

import os
tmp = os.environ["BINDER_REF_URL"]
tmp = tmp.split("/")
tmp = tmp[len(tmp) - 1]
repo_name = tmp[:tmp.find(".")]

!datalad uninstall -r /home/jovyan/input_data/*
!git-annex unannex ~/input_data/*
!git-annex unannex ~/output_data/*
!git-annex unannex ~/source/*

# !rm -rf ~/input_data/*
# !rm -rf ~/output_data/*
# !rm -rf ~/source/*

!rm -r ~/input_data/*
!rm -r ~/output_data/*
!rm -r ~/source/*

!scp -r mdx:~/$repo_name/input_data/* ~/input_data/
!scp -r mdx:~/$repo_name/output_data/* ~/output_data/
!scp -r mdx:~/$repo_name/source/* ~/source/
    
print('============================================================')
print('mdx環境からの実験データと実験実行コードの転送が完了しました。')

## 3. 実験記録をデータガバナンス機能に同期する

実験記録をデータガバナンス機能に同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

# Git管理のパスのリストを作成する
%cd ~/
files = os.listdir()
# ディレクトリ一覧からGit-annex管理するディレクトリ(input_dataとoutput_data)を排除する
dirs = [f for f in files if os.path.isdir(f)]
dirs.remove('input_data')
dirs.remove('output_data')
# HOME直下のファイルを取得
files = [f for f in files if os.path.isfile(f)]
# Git管理するパスの配列を作成する
files.extend(dirs)
save_path = files

# Git-annex管理するパスの配列を作成する
annexed_save_path = ['input_data', 'output_data']

In [ ]:
import papermill as pm

%cd ~/
# Git-annex管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験終了 (1/2)', PATH = annexed_save_path, IS_RECURSIVE = False)
)

In [ ]:
import papermill as pm

%cd ~/
# Git管理ファイルを保存
pm.execute_notebook(
    'EX-WORKFLOW/util/base_datalad_save_push.ipynb',
    '/home/jovyan/.local/push_log.ipynb',
    parameters = dict(SAVE_MESSAGE = '実験終了 (2/2)', TO_GIT = True, PATH = save_path, IS_RECURSIVE = False)
)

## 4. この実験実行環境を停止・削除する

不要になったこの実行環境を停止・削除します。  
コード付帯機能では20までしか実行環境を構築できないため、不要な実行環境は削除することを推奨します。

### 4.1 この実行環境を確認する

以下のセルを実行して実行環境のサーバー名を確認して下さい。  

In [ ]:
import os

print(os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3])

### 4.2 コード付帯機能のコントロールパネルへ遷移し、実行環境を停止・削除する

[コントロールパネル](https://jupyter.cs.rcos.nii.ac.jp/hub/home)へ遷移して、`4.1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。  
※`delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。  
![コンテナ削除キャプチャ](https://raw.githubusercontent.com/ivis-kuwata/workflow-template/develop/sections/PACKAGE/EX-WORKFLOW/images/コンテナ削除キャプチャ.png)